# HD Wallet

## TOC:
* [Entropy](#entropy)
* [Mnemonic creation](#mnemonic-create)

In [649]:
from cryptography.hazmat.primitives import hashes
from bitarray import bitarray, util
from secrets import token_bytes

## Entropy <a clas="anchor" id="entropy"></a>
Generate a blob between 128 and 256 bits from a secure random source.

In [650]:
def entropy_gen(n = 128):
    if n < 128 or n > 256:
        raise Exception("Entropy size must be between 128 and 256 bits inclusive.")
    if n % 32 != 0:
        raise Exception("Entropy size must be multiple of 32.")
    return token_bytes(n // 8)

print(entropy_gen(32*8).hex()) #256 bits

d00722c931d8eb4b926cad75ebe5f891e78c61a8c6a0a582cd9a69bdddcbf6a5


## Mnemonic creation <a class="anchor" id="mnemonic-create"></a>

In [651]:
def mnemonic(n = 128):
    entropy_bits = bitarray()
    entropy_bits.frombytes(entropy_gen(n))
    
    hash256 = hashes.Hash(hashes.SHA256())
    hash256.update(entropy_bits)
    bin_h = bitarray()
    bin_h.frombytes(hash256.finalize())
    
    entropy_bits = entropy_bits + bin_h[:n // 32]

    numbers = []
    for i in range(len(entropy_bits)//11):
        numbers.append(util.ba2int(entropy_bits[11*i:11*i+11]))

    words = {}
    with open('BIP-0039_english.txt', 'r') as file:
        for i, word in enumerate(file):
            if i in numbers:
                words[i] = word
                if len(numbers) == len(words):
                    break
    
    mnemonic_sentence = []
    for i in numbers:
        mnemonic_sentence.append(words[i].strip()) 
    
    return tuple(mnemonic_sentence)
    


print(mnemonic())


('fault', 'side', 'science', 'pen', 'message', 'link', 'visa', 'stem', 'advance', 'decade', 'day', 'drip')
